### 1. General info of dataset GSE148218

This is the Jupyter Notebook for dataset GSE148218. Its dataset includes barcodes/features/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/genes/matrix files and generate an AnnData object for each sample. 
In total, there are 8 acute lymphoblastic leukemia (ALL) samples.

In [2]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes

1. `barcodes.tsv.gz`: cell barcodes, which go into `.obs`
2. `features.tsv.gz`: gene names, `.var`
3. `matrix.mtx.gz`: the expression matrix, `.X`

In [8]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE148218')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE148218')

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    sample = sc.read_10x_mtx(
    sample_directory,
    var_names='gene_symbols',  
    cache=False
    )

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'acute_lymphoblastic_leukemia'
    obs_metrics['dataset'] = 'GSE148218'
    obs_metrics['tissue'] = 'bone_marrow'
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    sample.obs = obs_metrics
    sample.obs.set_index("uni_barcode", drop=False, inplace=True)
    print(sample)

    # save the anndata object
    output_path = write_directory / sample_h5ad
    sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 6106 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7228 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4163 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7826 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4224 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7160 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7917 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_

In [4]:
# Add "sample_barcode" info for each sample
from pathlib import Path

# Specify directory paths
rdm_directory = Path('/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE148218')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE148218')

# Loop through all files in the directory
for file in rdm_directory.iterdir():
    if "_uni.h5ad" in file.name:        
        sample = anndata.read_h5ad(file)

        sample.obs['sample_barcode'] = file.stem.split("_uni")[0]
        print(sample)

        # save the anndata object
        output_path = write_directory / file.name
        sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 6123 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4163 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7826 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7917 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 7160 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4224 × 32738
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode', 'sample_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object w

### 3. Confirmation of created AnnData objects

In [7]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE148218')
sample_barcodes = set()

# Loop through all files in the directory
for file in write_directory.iterdir():
    if "_uni.h5ad" in file.name:
        sample = anndata.read_h5ad(file)
        sample_barcodes.update(set(sample.obs['sample_barcode']))

print(sample_barcodes)
print(len(sample_barcodes))

{'GSM4456257_ALL10-d15', 'GSM4456254_ALL9', 'GSM4456255_ALL10', 'GSM4456252_ALL3', 'GSM4456256_ALL12', 'GSM4456251_ALL1', 'GSM4456253_ALL8', 'GSM4456258_ALL12-d15'}
8


### 4. Convert AnnData objects to SingleCellExperiment objects

In [8]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE148218')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in sample_name:
        sample_anndata = anndata.read_h5ad(file)
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

In [ ]:
print()